In [1]:
# import libraries

from openai import OpenAI
import os
from dotenv import load_dotenv


In [2]:
from pathlib import Path

import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o-mini") 

##change everytime
extracted_raw_file = Path(r"C:\Users\herms\Desktop\Civic_Tech_Research_Project\PreliminarWork\preprocessing\work_on_minutes\acta_26_abril_2019\extracted_26_04_19\extracted_text_nh_nf_26_04_19_clean_vers_001.txt")

text_content = []

with open(extracted_raw_file, "r", encoding="utf-8") as text_file:
    text_full = text_file.read()
    
    # Count the number of tokens    
    instruction_tokens = encoding.encode(text_full)
    num_tokens = len(instruction_tokens)
    print(f"Number of tokens : {num_tokens}")

    text_content.append(text_full) #save for future processing
     
    print("\n ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")  
        

Number of tokens : 23307

 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [ ]:
## max input tokens 128K 
# if needed, for a long document, it is going to be necessary to break the document in pieces and make a recursive summarization 

In [21]:
#token encoding for the instruction

# Load the encoding for the specific model you're using
encoding = tiktoken.encoding_for_model("gpt-4o-mini")    

# Instruction text
instruction = f"Has un resumen de una minuta de reunión de un gobierno local.\
La intención de este ejercicio es proveer al ciudadano común con información relevante sobre el desempeño de su gobierno local. \
Instrucciones especificas: \
- Enfatiza los problemas, peticiones y soluciones o acuerdos que se llegan \
- Indica los nombres de las personas que participan cuando sea necesario.\
- Indica datos concretos \
- No emitas juicios de valor, manten neutralidad\
- Identifica fechas\
- Identifica los temas que se trataron.\
- Verifica con cuidado al final cuantas personas asistieron a la sesion\
"

# Encode the text
instruction_tokens = encoding.encode(instruction)

# Count the number of tokens
num_tokens = len(instruction_tokens)

print(f"Number of tokens: {num_tokens}")


Number of tokens: 124


In [22]:
# loading API key
load_dotenv()
key_ct = os.environ["ct_api_key"]

In [29]:

client = OpenAI(api_key = key_ct)

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content" : instruction},
        {"role":"user", "content":text_content[0]},
    ],
    temperature = 0.6,
    max_tokens = 1000,
)


In [30]:

#change everytime
response_summary_TOTAL_vers005 = response.choices[0].message.content
print(response_summary_TOTAL_vers005)   

# Resumen de la Sesión Extraordinaria del Concejo Metropolitano de Quito

**Fecha:** 26 de abril de 2019  
**Hora de inicio:** 14:50  
**Asistentes:** 11 concejales y el Alcalde, Dr. Mauricio Rodas Espinel.  
**Total de asistentes:** 13 personas.

## Temas tratados:

### 1. Comparecencia de la Gerencia de EMGIRS
- **Problema:** El concejal Marco Ponce expresó su preocupación por la ausencia del gerente de EMGIRS en la sesión anterior, lo que considera un desacato a la resolución del Concejo.
- **Acuerdo:** El Alcalde se comprometió a incluir la comparecencia del gerente en la próxima sesión ordinaria del Concejo.

### 2. Proyecto de Ordenanza sobre Incremento de Pisos
- **Descripción:** Se discutió la Ordenanza Metropolitana que regula el incremento de pisos por suelo creado, en proyectos eco-eficientes ubicados en áreas de influencia del Sistema Metropolitano de Transporte y en Zonas Urbanísticas de Asignación Especial (ZUAE).
- **Participantes:** La Comisión de Uso de Suelo, con la p

## Exporting to .txt 

In [31]:

#change everytime
with open(r"C:\Users\herms\Desktop\Civic_Tech_Research_Project\PreliminarWork\preprocessing\work_on_minutes\acta_26_abril_2019\summary_26_04_19\response_summary_TOTAL_vers005.txt", "w", encoding="utf-8") as text_file:
 
    text_file.write(response_summary_TOTAL_vers005)  # Write the block text to the file, with spacing between blocks

print("Text extraction completed and saved'.")


Text extraction completed and saved'.


## Exporting to .md

In [6]:

#change everytime
with open(r"C:\Users\herms\Desktop\Civic_Tech_Research_Project\PreliminarWork\preprocessing\work_on_minutes\acta_26_abril_2019\summary_26_04_19\response_summary_TOTAL_vers002.md", "w", encoding="utf-8") as text_file:
 
    text_file.write(response_summary_TOTAL_vers002)  # Write the block text to the file, with spacing between blocks

print("Text extraction completed and saved to 'response_summary_TOTAL_vers002'.")


Text extraction completed and saved to 'response_summary_TOTAL_vers002'.
